<a href="https://colab.research.google.com/github/nikhiltarte/Llama3_ChatInterface/blob/main/Llama_3_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing the Dependencies**

In [2]:
!pip install -r requirements.txt
# acceLerate==0.29.3
# bitsandbytes==0.43.1
# transformers==4.40.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

In [3]:
import json
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)

**HF account Configuration**

In [6]:
config_data = json.load(open("config.json"))
HF_TOKEN = config_data["HF_TOKEN"]

In [8]:
model_name = "meta-llama/Meta-Llama-3-8B"

**Quantisation Configuration**

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

**Loading the Tokenizer and the LLM**

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=HF_TOKEN)

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [12]:
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128
)

In [13]:
def get_response(prompt):
  sequences = text_generator(prompt)
  gen_text = sequences[0]["generated_text"]
  return gen_text

In [14]:
prompt = "What is Machine Learning?"

In [15]:
llama3_response = get_response(prompt)

In [16]:
llama3_response

'What is Machine Learning? How Does It Work?\nWhat is Machine Learning? How Does It Work?\nIn today’s world, machine learning is a term that is becoming increasingly more common. However, what exactly does it mean? And how does it work? In this blog post, we will explore the answers to these questions. We will discuss what machine learning is and how it works. We will also look at some of the benefits of machine learning and how it is being used in today’s world. So, if you’re curious about machine learning, keep reading!\nMachine learning is a type of artificial intelligence that allows computers to learn from data. In other words,'

In [17]:
# prompt: Write a code to create the chat interface in loop, get out of loop once write exit in the input text feild.

while True:
  prompt = input("Enter your prompt (or type 'exit' to quit): ")
  if prompt.lower() == "exit":
    break
  response = get_response(prompt)
  print(response)


Enter your prompt (or type 'exit' to quit): Tell me about Paraglyding
Tell me about Paraglyding
How long have you been involved with the sport?
I’ve been paragliding for 7 years.
What is your favorite part about paragliding?
Flying. I love flying. I love being able to take off from anywhere, and fly anywhere. I love the feeling of flying, and the freedom it gives you.
Do you have a favorite paragliding location?
I have a few. I love flying in the mountains of Colorado, but I also love flying in the mountains of California. I also love flying in the mountains of New Mexico. I love flying in the mountains of Utah. I love flying in
Enter your prompt (or type 'exit' to quit): 3+5
3+5=8 and 2+6=8. This means that 8 is a sum of two positive integers, and also a sum of two negative integers. Thus, the sum of these two numbers is zero. The same happens with the numbers 5 and -3. The sum of these two numbers is zero.
If you add a positive number and a negative number, you get a sum that is equa

In [ ]:
print(llama3_response)

What is Machine Learning? Machine learning is a branch of artificial intelligence that deals with the development of algorithms that can learn from data and make predictions. Machine learning algorithms are used in a variety of applications, including image recognition, natural language processing, and fraud detection. Machine learning is a rapidly growing field, with new algorithms and applications being developed all the time. Machine learning is a branch of artificial intelligence that deals with the development of algorithms that can learn from data and make predictions. Machine learning algorithms are used in a variety of applications, including image recognition, natural language processing, and fraud detection. Machine learning is a rapidly growing field, with new algorithms and applications being developed


In [ ]:
print(llama3_response[len(prompt):])

 Machine learning is a branch of artificial intelligence that deals with the development of algorithms that can learn from data and make predictions. Machine learning algorithms are used in a variety of applications, including image recognition, natural language processing, and fraud detection. Machine learning is a rapidly growing field, with new algorithms and applications being developed all the time. Machine learning is a branch of artificial intelligence that deals with the development of algorithms that can learn from data and make predictions. Machine learning algorithms are used in a variety of applications, including image recognition, natural language processing, and fraud detection. Machine learning is a rapidly growing field, with new algorithms and applications being developed
